# Kenya 2022 Drought Evaluation using AI Weather Reforecasts
This notebook evaluates the 2022 drought over Kenya using reforecasts from Pangu-Weather, GraphCast, and AIFS,
with IMERG precipitation as the observational reference.

**Variables**: Precipitation & Soil Moisture

**Period**: January 2022 – December 2022

In [ ]:
import xarray as xr

In [ ]:
# 1. Setup and Imports
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
from datetime import datetime
import requests

In [ ]:
# 2. Define Region and Time
KENYA_BBOX = [33.5, 42.0, -5.0, 5.5]  # [lon_min, lon_max, lat_min, lat_max]
DATE_RANGE = pd.date_range('2022-01-01', '2022-12-31', freq='MS')

## 3. Download IMERG Data via NASA GES DISC API
You'll need a NASA Earthdata Login (https://urs.earthdata.nasa.gov/).

**Register and generate a `.netrc` file**:
```
machine urs.earthdata.nasa.gov
login YOUR_USERNAME
password YOUR_PASSWORD
```

In [ ]:
# 3.1 Example: Download IMERG monthly data from GES DISC
def download_imerg_monthly(year, month, out_dir="imerg_data"):
    os.makedirs(out_dir, exist_ok=True)
    base_url = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGM.06"
    date_str = f"{year}{month:02d}01"
    filename = f"3B-MO.MS.MRG.3IMERG.{date_str}-S000000-E235959.{year}{month:02d}01.V06B.HDF5"
    url = f"{base_url}/{year}/{filename}"
    output_path = os.path.join(out_dir, filename)
    print(f"Downloading: {url}")
    
    with requests.get(url, stream=True, auth=("YOUR_USERNAME", "YOUR_PASSWORD")) as r:
        r.raise_for_status()
        with open(output_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"Saved to {output_path}")

# Example call for January 2022
# download_imerg_monthly(2022, 1)

## 4. Open and Plot IMERG Data

In [ ]:
import h5py

def load_imerg_file(filepath):
    with h5py.File(filepath, 'r') as f:
        precip = f['Grid/precipitation'][()]
        lat = f['Grid/lat'][:]
        lon = f['Grid/lon'][:]
    return xr.DataArray(precip, coords=[lat, lon], dims=["lat", "lon"])

# Example usage (after download)
# da = load_imerg_file("imerg_data/3B-MO.MS.MRG.3IMERG.20220101-S000000-E235959.20220101.V06B.HDF5")
# da.plot(cmap='YlGnBu'); plt.title("IMERG Precipitation (Jan 2022)"); plt.show()

## 5. Load Pangu-Weather Forecast Data from AWS Open Data

In [ ]:
import s3fs

def load_pangu_forecast(date_str="20220701", hour="00"):
    fs = s3fs.S3FileSystem(anon=True)
    prefix = f"noaa-oar-mlwp-data/PANG_v100_GFS/{date_str}/"
    files = fs.ls(prefix)
    file = next(f for f in files if f"{date_str}{hour}_f000_f240" in f and f.endswith(".nc"))
    print(f"Loading: s3://{file}")
    with fs.open(file, 'rb') as f:
        ds = xr.open_dataset(f)
    return ds

# Example usage
# ds_pangu = load_pangu_forecast("20220701", "00")
# ds_pangu['prcp'].isel(time=0).plot(cmap='Blues'); plt.title("Pangu Precipitation Forecast"); plt.show()